In [4]:
import matplotlib
matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np

from utility_match_units import (binary_reader_waveforms,match_units,
                     search_spikes_parallel, load_ks2_spikes,
                     reload_ks2_templates, Match_to_ground_truth,
                     plot_single_unit)

clrs = ['blue', 'red', 'green', 'cyan',
'black','grey','brown','slategrey','darkviolet','darkmagenta',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']


Autosaving every 180 seconds


In [5]:
# dataset aprameters:
root_dir = ('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/synthetic_drift/run9/')
n_channels = 384
n_times = 101

# convert KS2 spike trains to 2 column standard and reorder spikes by sequential index
spike_train_sorted = load_ks2_spikes(root_dir, n_channels, n_times)
#print ("Spike train: \n", spike_train_sorted)

# remake templates from data
print ("Remaking templates ")
data_type='int16'
templates_sorted = reload_ks2_templates(root_dir, spike_train_sorted, data_type, 
                                        n_channels, n_times)
print ("templates sorted: ", templates_sorted.shape)


FileNotFoundError: [Errno 2] No such file or directory: '/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/synthetic_drift/run9/spike_times.npy'

In [3]:
# match to ground truth data;
matcher = Match_to_ground_truth(root_dir, spike_train_sorted, templates_sorted)
print ("n_spikes: ", matcher.n_spikes)
print (" DONE MATCHING ")
matcher.n_times = n_times
matcher.n_channels = n_channels

 ground truth templates:  (100, 101, 384)
units_ground_truth:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
units_ground_truth:  [17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33]
units_ground_truth:  [34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50]
units_ground_truth:  [51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67]
units_ground_truth:  [68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83]
searching match unit:  7
# spikes in sorted unit:  180
# matches:  0
searching match unit:  11
# spikes in sorted unit:  180
# matches:  0
searching match unit:  12
# spikes in sorted unit:  180
# matches:  0
searching match unit:  15
# spikes in sorted unit:  193
# matches:  0
searching match unit:  20
# spikes in sorted unit:  180
# matches:  0
searching match unit:  32
# spikes in sorted unit:  180
# matches:  0
searching match unit:  36
# spikes in sorted unit:  523
# matches:  3
searching match unit:  39
# spikes in sorted unit:  180
# matches:  0
searching match unit:  42
# spik

# matches:  0
searching match unit:  518
# spikes in sorted unit:  182
# matches:  0
searching match unit:  519
# spikes in sorted unit:  180
# matches:  0
searching match unit:  520
# spikes in sorted unit:  7829
# matches:  0
searching match unit:  521
# spikes in sorted unit:  338
# matches:  1
searching match unit:  522
# spikes in sorted unit:  17715
# matches:  0
searching match unit:  523
# spikes in sorted unit:  221
# matches:  0
searching match unit:  525
# spikes in sorted unit:  14385
# matches:  48
searching match unit:  526
# spikes in sorted unit:  414
# matches:  1
searching match unit:  528
# spikes in sorted unit:  179
# matches:  0
searching match unit:  530
# spikes in sorted unit:  2453
# matches:  5
searching match unit:  532
# spikes in sorted unit:  203
# matches:  0
searching match unit:  533
# spikes in sorted unit:  14990
# matches:  0
searching match unit:  535
# spikes in sorted unit:  10385
# matches:  6
searching match unit:  536
# spikes in sorted unit: 

In [4]:
# Make Pie Charts from results;
if True:
    n_matches=3
    matcher.make_pie_charts(n_matches)
    

n matches:  3
sizes: [6873   64   50]  , clrs:  ['blue', 'red', 'green']
sizes: [4914   50   37]  , clrs:  ['blue', 'red', 'green']
sizes: [8604 2633  436]  , clrs:  ['blue', 'red', 'green']
sizes: [17401   178   157]  , clrs:  ['blue', 'red', 'green']
sizes: [7972 4864  860]  , clrs:  ['blue', 'red', 'green']
sizes: [5676 3075   96]  , clrs:  ['blue', 'red', 'green']
sizes: [9092   41   27]  , clrs:  ['blue', 'red', 'green']
sizes: [9373  103   99]  , clrs:  ['blue', 'red', 'green']
sizes: [2409 1699  989]  , clrs:  ['blue', 'red', 'green']
sizes: [2755   31   29]  , clrs:  ['blue', 'red', 'green']
sizes: [9078   44   44]  , clrs:  ['blue', 'red', 'green']
sizes: [2189  354   61]  , clrs:  ['blue', 'red', 'green']
sizes: [8092   81   57]  , clrs:  ['blue', 'red', 'green']
sizes: [5308   53   46]  , clrs:  ['blue', 'red', 'green']
sizes: [4047   25   25]  , clrs:  ['blue', 'red', 'green']
sizes: [14873   119    71]  , clrs:  ['blue', 'red', 'green']
sizes: [2190   22   19]  , clrs:  ['

In [ ]:
scale_amplitude = 10.
n_matches = 3
save_fig = True
# plot ptp of spikes from different units:
# selected_unit = 15 #15 #47 #39 #21

units = np.unique(matcher.spike_train_gt[:,1])
for selected_unit in units: 
    plot_single_unit(root_dir, selected_unit, matcher, 
                  n_matches, clrs, scale_amplitude,
                  save_fig)

matched_units:  [  7  11  12  15  20  32  36  39  42  46  50  51  56  57  67  68  78  84
  86  88  90 101 106 113 118 130 135 139 141 142 148 160 174 201 211 213
 217 218 226 228 230 231 233 235 238 241 244 249 254 262 267 270 277 278
 283 288 291 305 307 311 313 316 317 318 334 336 342 344 350 351 353 354
 360 361 363 368 370 371 375 379 382 385 397 414 417 441 443 445 455 461
 479 486 493 494 495 496 497 499 500 501 504 506 508 511 513 514 515 516
 517 518 519 520 521 522 523 525 526 528 530 532 533 535 536 538 540 543
 547 552 555 556 557 559 560 565 566 567 568 572 574 575 576 577 580 585
 593 599 601 604 609 612 620 622 629 630 632 648 649 651 652 655 656 659
 665 669 672 673 674 677 679 685 690 701 703 710 713 715 716 727 735 737
 740 745 751 763 765 767 776 777 778 782 789 790 791 792 800 805 808 809
 811 812 816 822 826 830 836]
 unit:  0 , matching unit:  101 506

 unit:  0 , matching unit:  92 493

 unit:  0 , matching unit:  131 559

 loading injected unit:  0
 ground truth 


 unit:  8 , matching unit:  133 594

 unit:  8 , matching unit:  168 719

 loading injected unit:  8
 ground truth wfs:  (5112, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  3  18  25  49  58  60  61  72  73  77  95  98  99 105 112 115 117 120
 126 128 137 149 152 159 170 185 188 193 195 199 200 236 251 256 257 269
 273 274 276 285 286 289 292 299 300 303 304 306 312 314 319 320 321 322
 323 324 325 327 328 329 332 333 337 339 341 343 345 346 348 349 355 357
 359 362 364 365 366 369 373 374 377 380 381 386 387 389 390 391 392 393
 396 400 403 404 407 408 409 411 413 415 419 422 425 428 453 456 477 485
 503 505 507 512 531 534 542 579 586 591 597 602 616 624 634 638 658 662
 666 689 692 700 704 705 707 717 728 738 739 741 748 753 756 761 775 785
 794 796 815 817 827]
 unit:  9 , matching unit:  72 359

 unit:  9 , matching unit:  52 321

 unit:  9 , matching unit:  43 299

 loading injected unit:  9
 ground truth wfs:  (2761, 101, 384)
len ptpssorted:  3
rec len se

In [ ]:
print (len(ptp_gt_unit), len(purity),len(completeness))
print (ptp_gt_unit)
print (purity)
ax=plt.subplot(121)
plt.scatter(ptp_gt_unit, purity, c='blue')
plt.title("True Positive Rate \n(# spikes found / # spikes injected)", fontsize=15)
plt.xlabel("PTP of unit", fontsize=20)
plt.xlim(left=0)
plt.ylim(0,1.05)
ax.tick_params(axis='both', which='major', labelsize=20)

# plot purity
ax=plt.subplot(122)
ax.tick_params(axis='both', which='major', labelsize=20)
plt.scatter(ptp_gt_unit,completeness, c='red')
plt.title("Purity \n(# spikes found / # spikes in unit)", fontsize=15)
plt.xlabel("PTP of injected unit", fontsize=20)
plt.xlim(left=0)
plt.ylim(0,1.05)
plt.suptitle("Neuropixels Drift Simulation run3\n shifts: "+
             str(np.array([-0.1,  -0.05,  0.,    0.05])*40)+" um"+ " @ "+
             str(np.array([0.,  6000000., 12000000., 18000000.])/30000)+ " sec"+
            "\n10mins recording, 384 channels"+
            "\n Sorted using KS2")
plt.show()

In [ ]:
t1 = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/templates_ground_truth.npy')
print (t1.shape)
t2 = np.load('/media/cat/1TB/data/synthetic/run3/templates_reloaded_good.npy')
print (t2.shape)

max1 = t1[2].ptp(0).argmax(0)
max2 = t2[50].ptp(0).argmax(0)
print (max1, max2)

ax=plt.subplot(121)
plt.plot(t1[2],c='black')
ax=plt.subplot(122)
plt.plot(t2[50]/10.,c='blue')
plt.show()


In [ ]:
d1 = max_chans_sorted
d2 = max_chans_unit
t1, t2, t3 = np.intersect1d(d1,d2,return_indices=True)
print (t1,t2,t3)

t1 = np.where(np.in1d(d1, d2))[0]
print (t1)



In [67]:
spike_train = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/spike_train_ground_truth.npy')
print (spike_train)
templates = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/templates_ground_truth.npy')
print (templates.shape)

units = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/units.npy')
print (units)

ctr=0
spike_train_corrected = spike_train.copy()
for unit in units:
    idx = np.where(spike_train[:,1]==unit)[0]
    spike_train_corrected[idx,1]=ctr
    ctr+=1
    
print (spike_train_corrected)
np.save('/media/cat/1TB/data/synthetic/run3/ground_truth/spike_train_ground_truth_corrected.npy',spike_train_corrected)

[[      35      420]
 [    6119      420]
 [   12204      420]
 ...
 [17983395      247]
 [17990820      247]
 [17998245      247]]
(49, 101, 384)
[420 339 421 348 473 418  62 362 565 239 498 474 324 298 471 441 417 443
 356 424  63 437 485 160 156 302 346 334 431 486  66 204  14 258 300  16
 150  52 317 277 184 456  65 481  38 436 558 194 247]
[[      35        0]
 [    6119        0]
 [   12204        0]
 ...
 [17983395       48]
 [17990820       48]
 [17998245       48]]


In [21]:
units=np.arange(5)
units = [good_units[3]]

for unit in units:
    fig = plt.figure()
    ax = plt.subplot(131)
    #plt.plot(templates[unit].T)
    plt.plot(wfs_array[unit].mean(0))

    ax = plt.subplot(132)
    #spikes = spikes[idx]
    spikes = times[unit]
    print ("spikes:", spikes.shape, "ptps: ", ptps[unit].shape)
    plt.scatter(spikes/30000., ptps[unit])

    ax=plt.subplot(133)
    max_chan = wfs_array[unit].mean(0).ptp(0).argmax(0)#templates[unit].ptp(0).argmax(0)
    print (wfs_array[unit].shape)

    cmap = cm.get_cmap('viridis',wfs_array[unit].shape[0])
    clrs = cmap(np.arange(wfs_array[unit].shape[0]))
    print (clrs)
    for k in range(0,wfs_array[unit].shape[0],10):
        plt.plot(wfs_array[unit][k].T,c=cmap(k))


plt.show()



spikes: (8210,) ptps:  (8210,)
(8210, 101)
[[0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 ...
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]]


In [9]:
fname = '/home/cat/Downloads/yass/samples/10chan/phy/whitening_mat.npy'

n_channels = 10

whitening_mat = np.random.rand(n_channels,n_channels)

np.save(fname, whitening_mat)
print (whitening_mat.shape)
print (whitening_mat)

(10, 10)
[[0.61659694 0.20125714 0.48515898 0.45059603 0.89409843 0.8158124
  0.30200264 0.06808628 0.60955651 0.7760584 ]
 [0.19814057 0.21134196 0.27791917 0.52312241 0.20263871 0.21097603
  0.34816364 0.1852993  0.90506002 0.80562087]
 [0.970196   0.70466193 0.08746122 0.33371635 0.44791585 0.00678562
  0.41421277 0.65443008 0.8541354  0.41522301]
 [0.88052786 0.34072116 0.12298287 0.23288985 0.98943396 0.56546028
  0.38736253 0.87851743 0.83882004 0.96018176]
 [0.5622632  0.19456103 0.89831444 0.56587043 0.12599613 0.24817026
  0.47188959 0.55809954 0.72628178 0.72821972]
 [0.59236306 0.06332327 0.35662913 0.93425548 0.15622572 0.77164442
  0.62013196 0.67339586 0.97380542 0.9211003 ]
 [0.55011567 0.22242238 0.41892959 0.23396425 0.80036351 0.00612786
  0.72244072 0.1119026  0.84247935 0.13666388]
 [0.1674969  0.17014984 0.70297569 0.28534628 0.87704159 0.62894665
  0.38268201 0.47207143 0.53495952 0.46683818]
 [0.58624526 0.67612232 0.60880662 0.66169478 0.10866143 0.88197916
  0.

In [24]:
max_chans = templates.ptp(1).argmax(1)
ptps = templates.ptp(1).max(1)
ptps_all = templates.ptp(1)
print (ptps_all.shape)

print (max_chans)
print (ptps)


(426, 364)
[  0   1   1   1   2   3   4   6   7   5  10  13  14  14  15  16  17  18
  20  21  30  26  29  29  30  31  32  35  37  41  42  43  45  46  49  50
  38  56  58  60  69  68  68  68  72  70  71  74  75  76 342  80  79  80
  80  83  82  84 333  85  86  87  88  88  88  88  89  89  90  88  87  92
 336  94  95  98  99  84 101 103 104 108 109 109 111 111 111 112 114 115
 119 119 120 121 125 127 127 124 131 133 134 134 132 140 137 138 140 332
 140 140 141 142 142 140 143 145 146 147 147 149 149 149 149 150 284 149
 154 154 154 162 164 165 166 164 169 170 173 173 174 177 178 173 180 181
 182 186 182 185 185 186 186 189 190 192 194 197 198 199 201 204 204 206
 208 208 209 209 213 209 210 212 205 209 213 213 213 213 216 216 216 217
 219 220 226 228 229 229 229 230 230 233 235 215 236 236 238 239 239 242
 245 245 248 248 248 248 249 249 252 252 253 253 262 262 262 262 262 262
 262 262 262 262 262 262 252 265 265 265 266 266 257 268 268 253 273 273
 274 277 277 277 280 262 280 282 280 284

In [33]:
# plot scatter vs. depth
fig=plt.figure()
ax=plt.subplot(111)
for k in range(426):
    idx = np.where(spike_clusters==k)[0]
    times = spike_times[idx]/30000.
    plt.scatter(times, times*0+geom[max_chans[k],1]+(np.random.rand()*80-40),s=1, alpha=.9)
    
ax.tick_params(axis = 'both', which = 'major', labelsize = 20)
plt.ylim(3800,-100)
plt.xlim(-10,times[-1]+10)
plt.ylabel("Depth (um)",fontsize=20)
plt.xlabel("Time (sec)",fontsize=20)

plt.show()



In [59]:
import wavefile

data = wavefile.loadWave('/home/cat/Downloads/time code signal.wav')

print (data[1].shape)

plt.plot(data[1].squeeze()[:1000])
plt.show()

(1, 200000)


In [70]:
datas = data[1].squeeze()
print (datas.shape)
print (datas)

(200000,)
[ 0.5  0.5  0.5 ... -0.5 -0.5 -0.5]


In [74]:
idx = np.where(datas>0)[0]
print (idx)
diffs = idx[1:]-idx[:-1]
print (diffs)

[     0      1      2 ... 199992 199993 199994]
[1 1 1 ... 1 1 1]


In [75]:
start = []
end = []
start.append(0)
for k in range(1, diffs.shape[0]-1, 1):
    if (diffs[k]-diffs[k-1])>1:
        end.append(k-1)
        start.append(k)
end.append(k)
end = np.array(end)
start = np.array(start)
print ((end-start)[:100])
diffs_redone = end-start+1
print (diffs_redone)

[ 7 10  8  9  9  9  9  9  9  9  9  9  9  9  4  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  4  4  4  4  4  4  4  4  4  4  4  4  4  9  4
  9  9  9  9]
[ 8 11  9 ...  5  5  4]


In [79]:
ax=plt.subplot(111)
sample_rate = 20000.
hist = np.histogram(diffs_redone, np.arange(-20,50,1))
plt.bar(hist[1][:-1]/sample_rate,hist[0],0.9/sample_rate)
plt.xlabel("Inter pulse interval (seconds)")
plt.title("Distribution of inter-spike-interval (20 sec wav file provided by Avisoft)")
#ax.set_yscale('log')
plt.show()